# Contact Information: Daniel Wieferich (dwieferich@usgs.gov)                                                                                                                                                                                                                       

###### Date: 20170526

# We were having issues with inserting complex geometries to the mapcentia gc2 instance through the sql API.  To help identify limitations with the sql API, this script tests the insert capabilities and tracks the number of vertices a geometry can have before the insert fails.

In [ ]:
import geopandas as gpd
import getpass
import pandas as pd
import numpy as np
import requests

#Defines directories and credent
execfile('theStuff.py')

In [ ]:
#Imports a shapefile of 1500 line segments with various numbers of vertices.  Imports as geodataframe.
lin1 = directory + "/Drainage/nhdflowline1500.shp"
gdf2 = gpd.read_file(lin1)

In [8]:
d = []
for index, row in gdf2.iterrows():
    line = str(row['geometry'])  
    line = line.replace("(", "((")
    line = line.replace(")", "))")
    
    #Counts number coordinate pairs.  Counts spaces and divides that by 2 to get number of coordinate pairs (roughly).  
    #This doesn't account for text at beginning but using this just for an approximation so don't want to waste processing time refining text 
    countCoor = str(len(line.split())/2)
    
    #Note that gc2 forced geojson input to be a multilinestring, the data is actually linestring, geometry value needs to be changed to Multilinestring which is reflected below
    #note gc2key is coming from 'theStuff.py' as variable
    query = "https://gc2.mapcentia.com/api/v1/sql/bcb?q=insert into sfr.line_test2 (comid, the_geom) VALUES ('" + str(row['COMID']) + "', ST_GeomFromText('MULTI" + line + "',4269))&key=" + gc2k
    
    
    #This should use a retry for  requests or alike to handle with connection issues
    try:
        r = requests.get(query)
        status = str(r.status_code)
        if r.status_code == 200:
            d.append({"coordCnt": countCoor, 'result': 1, 'status': status, 'comid': str(row['COMID'])})
        else:
            d.append({"coordCnt": countCoor, 'result': 2, 'status': status, 'comid': str(row['COMID'])})
            print str(row['COMID'])
    except:
        d.append({"coordCnt": countCoor, 'result': 3, 'status': 'didnt work', 'comid': str(row['COMID'])})
        print "crap" + str(row['COMID'])
        continue
        

df = pd.DataFrame(d)
df.head()

6641118
6646624
6647026
6647062
6647204
6656933
6656993


,comid,coordCnt,result,status
0,6635568,25,1,200
1,6635570,81,1,200
2,6635572,14,1,200
3,6635574,14,1,200
4,6635576,172,1,200


In [44]:
#Printing things out all ugly like

print 'Comid         Count Coordinates        Response Status '
for index, row in df.iterrows():
    if  row['status'] != '200':
        print row['comid'], '           ', row['coordCnt'], '                       ', row['status']

Comid         Count Coordinates        Response Status 
6641118             195                         414
6646624             271                         414
6647026             250                         414
6647062             194                         414
6647204             293                         414
6656933             282                         414
6656993             293                         414


In [ ]:
#The next lowest  count that worked was 191 coordinate pairs so somewhere between 191 and 194 is the max